This notebook is a companion to the *N5 Basis Tutorial* on the [imglib2-blog](https://imglib.github.io/imglib2-blog/).

In [31]:
%mavenRepo scijava.public https://maven.scijava.org/content/groups/public
%maven org.scijava:scijava-common:2.97.0
%maven net.imglib2:imglib2:6.2.0
%maven org.janelia.saalfeldlab:n5:3.1.2
%maven org.janelia.saalfeldlab:n5-imglib2:7.0.0
%maven org.janelia.saalfeldlab:n5-universe:1.3.1

In [32]:
import java.nio.file.*;
import java.util.stream.*;
import java.util.concurrent.*;

import com.google.gson.*;

import net.imglib2.img.array.*;
import net.imglib2.util.*;

import org.janelia.saalfeldlab.n5.*;
import org.janelia.saalfeldlab.n5.imglib2.*;
import org.janelia.saalfeldlab.n5.universe.*;



In [33]:
//| output: false

// N5Factory can make N5Readers and N5Writers
var factory = new N5Factory();

// trying to open a container does not yet exist will throw an error 
// var n5Reader = factory.openReader("my-container.n5");

// creating a writer creates a container at the given location
// if it does not already exist
var n5Writer = factory.openWriter("my-container.n5");

// now we can make a reader
var n5Reader = factory.openReader("my-container.n5");

// test if the container exists
n5Reader.exists(""); // true

// "" and "/" both refer to the root of the container
n5Reader.exists("/"); // true

true

In [4]:
//| output: false
factory.openWriter("my-container.h5").getClass();   // N5HDF5Writer
factory.openWriter("my-container.n5").getClass();   // N5FSWriter
factory.openWriter("my-container.zarr").getClass(); // N5ZarrWriter

class org.janelia.saalfeldlab.n5.zarr.N5ZarrWriter

In [5]:
//| output: false
n5Writer.createGroup("foo");
n5Writer.createGroup("foo/bar");
n5Writer.createGroup("lorum/ipsum/dolor/sit/amet");

n5Writer.exists("lorum/ipsum");      // true
n5Writer.exists("not/a/real/group"); // false

false

In [6]:
//| output: false
n5Writer.list("");     // [lorum, foo]
n5Writer.list("foo");  // [bar]

[Ljava.lang.String;@7ab37745

In [7]:
Arrays.toString(n5Writer.deepList(""));

[data, lorum, lorum/ipsum, lorum/ipsum/dolor, lorum/ipsum/dolor/sit, lorum/ipsum/dolor/sit/amet, put-data-in-me, foo, foo/bar]

In [8]:
// the arguments
var img = ArrayImgs.floats(64,64);
var groupPath = "data"; 
var chunkSize = new int[]{32,32};
var compression = new GzipCompression();

// save the image
N5Utils.save(img, n5Writer, groupPath, chunkSize, compression);

In [9]:
var exec = Executors.newFixedThreadPool(4); // with 4 parallel threads
N5Utils.save(img, n5Writer, groupPath, chunkSize, compression, exec);

In [34]:
n5Writer.remove(groupPath);

true

In [ ]:
var chunkSize = new int[]{64,};
N5Utils.save(img, n5Writer, groupPath, chunkSize, compression);

In [29]:
try (Stream<Path> stream = Files.walk(Paths.get("my-container.n5/data"))) {
    stream.filter(Files::isRegularFile)
        .filter( p -> p.getFileName().toString().matches("[0-9]"))
            .forEach( x -> { try {  
                    System.out.println(String.format("%s is %d bytes", x, Files.size(x))); 
                } catch(IOException e){}
            });
}

my-container.n5/data/1/1 is 50 bytes
my-container.n5/data/1/0 is 50 bytes
my-container.n5/data/0/1 is 50 bytes
my-container.n5/data/0/0 is 36 bytes


In [10]:
//| output: false
var loadedImg = N5Utils.open(n5Writer, groupPath);
Util.getTypeFromInterval(loadedImg).getClass();      // FloatType
Arrays.toString(loadedImg.dimensionsAsLongArray());  // [64, 64]

[64, 64]

In [11]:
//| output: false
// overwrite our previous data
var img = ArrayImgs.unsignedBytes(2,2);
N5Utils.save(img, n5Writer, groupPath, chunkSize, compression);

// load the new data, the old data are no longer accessible
var loadedImg = N5Utils.open(n5Writer, groupPath);
Arrays.toString(loadedImg.dimensionsAsLongArray());  // [2, 2]

[2, 2]

In [12]:
// create a group inside the container (think: "folder")
var groupName = "put-data-in-me";
n5Writer.createGroup(groupName);

// attributes have names and values
// make an attribute called "date" with a String value
var attributeName = "date";
n5Writer.setAttribute(groupName, attributeName, "2024-Jan-01");

// get the value of the "date" attribute
n5Writer.getAttribute(groupName, attributeName, String.class);

2024-Jan-01

In [13]:
//| output: false
n5Writer.setAttribute(groupName, "a", 42);
var num = n5Writer.getAttribute(groupName, "a", double.class); // 42.0
var str = n5Writer.getAttribute(groupName, "a", String.class); // "42"

In [14]:
//| code-fold: true
class FunWithMetadata {
    String name;
    int number;
    double[] data;
    
    public FunWithMetadata(String name, int number, double[] data) {
        this.name = name;
        this.number = number;
        this.data = data;
    }
    public String toString(){
        return String.format( "FunWithMetadata{%s(%d): %s}", 
            name, number, Arrays.toString(data));
    }
};

In [15]:
var metadata = new FunWithMetadata("Dorothy", 2, new double[]{2.72, 3.14});
n5Writer.setAttribute(groupName, "metadata", metadata);

var loadedMetadata = n5Writer.getAttribute(groupName, "metadata", 
    FunWithMetadata.class);
loadedMetadata

FunWithMetadata{Dorothy(2): [2.72, 3.14]}

In [16]:
n5Writer.getAttribute(groupName, "/", JsonElement.class);

{"date":"2024-Jan-01","a":42,"metadata":{"name":"Dorothy","number":2,"data":[2.72,3.14]}}

In [17]:
// set attributes
n5Writer.setAttribute(groupName, "sender", "Alice");
n5Writer.setAttribute(groupName, "receiver", "Bob");

// notice that they're set
n5Writer.getAttribute(groupName, "sender", String.class);   // Alice
n5Writer.getAttribute(groupName, "receiver", String.class); // Bob

// remove "sender"
n5Writer.removeAttribute(groupName, "sender");

// remove "receiver" and store result in a variable
var receiver = n5Writer.removeAttribute(groupName, "receiver", String.class); // Bob

n5Writer.getAttribute(groupName, "sender", String.class);   // null
n5Writer.getAttribute(groupName, "receiver", String.class); // null

In [18]:
Arrays.toString(n5Writer.getAttribute("data", "dimensions", long[].class));

[2, 2]